## Asset Type 3
* Use Case Matrix:

| Type     | Transferable    | Quantifiable     | Examples, Samples, Use Cases                  |
|----------|-----------------|------------------|-----------------------------------------------|
| 3        | N               | Y                | Travel points, Redeemable Tokens              |
|          |                 |                  | Amazon Vouchers                               |

### Initialisation:
* The initialisation is for initialise the python scripts in the notebook get it connected to CorDapp.
* Expected to see a base directory path.

In [1]:
import os
import sys
import math
import time
import IPython
import logging
import warnings
import datetime
import pandas as pd
import numpy  as np
import webbrowser
import pprint as pp
from random   import randrange
from notebook import notebookapp
from IPython.display import display
from IPython.core.display import HTML

# Initialise the notebook settings
def init_notebook():
    sys.path.insert(1, os.path.join(sys.path[0], '{}/lib/'.format(os.environ['BaseDir'])))
    display(HTML("<style>.container { width:98% !important; }</style>"))
    %matplotlib inline
    
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)

# Initialise the Corda nodes
def init_cordaNodes():
    from corda5Interface import Corda5
    apiServer='https://localhost:8888/api/v1/flow'
    apiServerAuth=('admin', 'admin')
    package = 'com.r3.developers.configurableInstrument'
    replace_maters_list = { 'API_SERVER': apiServer, 'PACKAGE': package}

    nodes = {
            'Authority' : {'x500': 'CN=Authority, OU=Test Dept, O=R3, L=London, C=GB' ,'hash': 'F0E569C4765C' },
            'Bob'       : {'x500': 'CN=Bob, OU=Test Dept, O=R3, L=London, C=GB'       ,'hash': '972B13CEE093' },
            'Charlie'   : {'x500': 'CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB'   ,'hash': 'AD4AA4F0D9B1' },
            'Alice'     : {'x500': 'CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'     ,'hash': 'F07CC621F1DF' },
            }

    h = Corda5(level=logging.INFO, nodes=nodes, apiServer=apiServer, apiServerAuth=apiServerAuth, version=2)

    log = h.log
    log.info("Base directory {}".format(os.environ['BaseDir']))
    
    return h

init_notebook()
h = init_cordaNodes()

 19-05 09:35:51 | I | Base directory /Users/bo.yang/Documents/github/configurableLedgerAsset/python3


### Issue -- Type 3, Transferable = N, Quantifiable = Y
* Issuer gets to decide the name, transferable or not, expiry or not, redeemable or not, and can add multiple/additional attributes (HashMap) on the same State/Asset.
* Applications like Government Bond, will be transferable, with maturity(some expiry), and redeemable (cash the principal on maturity), with additional attribute of coupon payment dates.
* Applications like College University Degree, will not be transferable, no expiry, and not redeemable. Will have additional attributes like Collage Name, University Name, Education Stream etc etc along with Issuer and Owner(Holder).
* Applications like Limited Edition Sovereign, will be transferable, no expiry but will be redeemable directly with Issuer.

In Asset type 3, all the issuing should be **succeed**, with **Non-Transferable** and **Quantifiable** assets.

In [21]:
# Issue Test 1 -- Issue Travel Points / Redeemable Tokens 
# Expected -- Succeed
(req_id, response, return_val_uuid) = h.issue('Authority', 'Alice', 'Star Alliance Travel Points', 
        quantity=10,
        transferable=False,
        expiry=None,
        verifiable=True)
h.message([req_id, response, return_val_uuid])

 19-05 10:15:07 | I | Issuing instrument='Star Alliance Travel Points' to 'Alice'
 19-05 10:15:07 | I | Running action 'Issue' on 'Authority/F0E569C4765C/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 19-05 10:15:56 | I | d4ea67e0-9945-4fe5-b0da-c3ef0d457e32
 19-05 10:15:56 | I | Time taken = 0:00:48.491483


44156.1684462507.613406<Response [200]>d4ea67e0-9945-4fe5-b0da-c3ef0d457e32

In [24]:
# Issue Test 2 -- Issue Amazon Vouchers
# Expected -- Succeed
(req_id, response, return_val_uuid) = h.issue('Authority', 'Alice', 'Amazon Voucher', 
        quantity=200,
        transferable=False,
        expiry=None,
        verifiable=True,
        attributes={'currency': 'INR'})
h.message([req_id, response, return_val_uuid])

 19-05 10:17:21 | I | Issuing instrument='Amazon Voucher' to 'Alice'
 19-05 10:17:21 | I | Running action 'Issue' on 'Authority/F0E569C4765C/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 19-05 10:18:13 | I | c7a34000-ac8d-4d63-a0cf-ccb7b09435b2
 19-05 10:18:13 | I | Time taken = 0:00:52.511726


44156.1684462641.125758<Response [200]>c7a34000-ac8d-4d63-a0cf-ccb7b09435b2

### Query:
* Query is used for query the instruments that one/all held.

In [25]:
# Query Test 1 -- Query all the instruments
h.query_all_nodes()

 19-05 10:18:34 | I | Result for query=ListInstrument from 'Authority/F0E569C4765C/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.190050)


,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,d4ea67e0-9945-4fe5-b0da-c3ef0d457e32,Star Alliance Travel Points,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",10,False,None,True,{}
1,c7a34000-ac8d-4d63-a0cf-ccb7b09435b2,Amazon Voucher,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",200,False,None,True,{'currency': 'INR'}


 19-05 10:18:39 | I | Result for query=ListInstrument from 'Bob/972B13CEE093/CN=Bob, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.085144)


""


 19-05 10:18:43 | I | Result for query=ListInstrument from 'Charlie/AD4AA4F0D9B1/CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.087414)


""


 19-05 10:18:47 | I | Result for query=ListInstrument from 'Alice/F07CC621F1DF/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.086007)


,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,d4ea67e0-9945-4fe5-b0da-c3ef0d457e32,Star Alliance Travel Points,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",10,False,None,True,{}
1,c7a34000-ac8d-4d63-a0cf-ccb7b09435b2,Amazon Voucher,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",200,False,None,True,{'currency': 'INR'}


In [26]:
# Query Test 2 -- Query Alice's instruments
h.query('Alice')

,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,d4ea67e0-9945-4fe5-b0da-c3ef0d457e32,Star Alliance Travel Points,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",10,False,None,True,{}
1,c7a34000-ac8d-4d63-a0cf-ccb7b09435b2,Amazon Voucher,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",200,False,None,True,{'currency': 'INR'}


### Transfer:
* Not all instruments are transferable.
* If transferable -- will work on the basic nature of the instrument decided right from the issue point(by the issuer).
* Check = not expired apart from being owned.

In Asset Type 3, as it's non tranferable, all the transfers should be **failure**.

In [27]:
# Transfer Test 1 -- Partial transfer from Holder for instrument Type 3, Transferable = N, Quantifiable = Y
# Expected -- Failure
(req_id, response, return_val) = h.transfer(return_val_uuid, 'Alice', 'Charlie', quantity=1)
h.message([req_id, response, return_val])

 19-05 10:38:04 | I | Tranfering instrument id 'c7a34000-ac8d-4d63-a0cf-ccb7b09435b2' to 'Charlie'
 19-05 10:38:04 | I | Running action 'Transfer' on 'Alice/F07CC621F1DF/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'
 19-05 10:38:08 | I | {'type': 'FLOW_FAILED', 'message': 'net.corda.v5.ledger.utxo.ContractVerificationException: Verification of ledger transaction with ID SHA-256D:11A2F0F6AE340877D3D973F3EA44551561E99CACB294A84E78C885C125A06ED5 failed: net.corda.v5.ledger.utxo.ContractVerificationException: Ledger transaction contract verification failed for the specified transaction: SHA-256D:11A2F0F6AE340877D3D973F3EA44551561E99CACB294A84E78C885C125A06ED5.\nThe following contract verification requirements were not met:\ncom.r3.developers.configurableInstrument.contracts.InstrumentContract: Failed requirement: Instrument is not transferable.\n'}
 19-05 10:38:08 | I | Time taken = 0:00:04.163592


44156.1684463884.192985<Response [200]>{'type': 'FLOW_FAILED', 'message': 'net.corda.v5.ledger.utxo.ContractVerificationException: Verification of ledger transaction with ID SHA-256D:11A2F0F6AE340877D3D973F3EA44551561E99CACB294A84E78C885C125A06ED5 failed: net.corda.v5.ledger.utxo.ContractVerificationException: Ledger transaction contract verification failed for the specified transaction: SHA-256D:11A2F0F6AE340877D3D973F3EA44551561E99CACB294A84E78C885C125A06ED5.\nThe following contract verification requirements were not met:\ncom.r3.developers.configurableInstrument.contracts.InstrumentContract: Failed requirement: Instrument is not transferable.\n'}

In [29]:
# Transfer Test 2 -- Full transfer from Holder 
# Expected -- Failure
(req_id, response, return_val) = h.transfer(return_val_uuid, 'Alice', 'Charlie', quantity=200)
h.message([req_id, response, return_val])

 19-05 10:40:23 | I | Tranfering instrument id 'c7a34000-ac8d-4d63-a0cf-ccb7b09435b2' to 'Charlie'
 19-05 10:40:23 | I | Running action 'Transfer' on 'Alice/F07CC621F1DF/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'
 19-05 10:40:27 | I | {'type': 'FLOW_FAILED', 'message': 'net.corda.v5.ledger.utxo.ContractVerificationException: Verification of ledger transaction with ID SHA-256D:6A67FEA0148F3D0A9FA71E6A19AE63B72ED74555B42A9314EB1BD84A5DB7D2FA failed: net.corda.v5.ledger.utxo.ContractVerificationException: Ledger transaction contract verification failed for the specified transaction: SHA-256D:6A67FEA0148F3D0A9FA71E6A19AE63B72ED74555B42A9314EB1BD84A5DB7D2FA.\nThe following contract verification requirements were not met:\ncom.r3.developers.configurableInstrument.contracts.InstrumentContract: Failed requirement: Instrument is not transferable.\n'}
 19-05 10:40:27 | I | Time taken = 0:00:04.069765


44156.1684464023.852549<Response [200]>{'type': 'FLOW_FAILED', 'message': 'net.corda.v5.ledger.utxo.ContractVerificationException: Verification of ledger transaction with ID SHA-256D:6A67FEA0148F3D0A9FA71E6A19AE63B72ED74555B42A9314EB1BD84A5DB7D2FA failed: net.corda.v5.ledger.utxo.ContractVerificationException: Ledger transaction contract verification failed for the specified transaction: SHA-256D:6A67FEA0148F3D0A9FA71E6A19AE63B72ED74555B42A9314EB1BD84A5DB7D2FA.\nThe following contract verification requirements were not met:\ncom.r3.developers.configurableInstrument.contracts.InstrumentContract: Failed requirement: Instrument is not transferable.\n'}

In [30]:
# Transfer Test 3 -- Transfer from Issuer 
# Expected -- Failure
(req_id, response, return_val) = h.transfer(return_val_uuid, 'Authority', 'Charlie', quantity=100)
h.message([req_id, response, return_val])

 19-05 10:40:47 | I | Tranfering instrument id 'c7a34000-ac8d-4d63-a0cf-ccb7b09435b2' to 'Charlie'
 19-05 10:40:47 | I | Running action 'Transfer' on 'Authority/F0E569C4765C/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 19-05 10:40:51 | I | {'type': 'FLOW_FAILED', 'message': 'Not able to find a owning state with id c7a34000-ac8d-4d63-a0cf-ccb7b09435b2'}
 19-05 10:40:51 | I | Time taken = 0:00:04.073150


44156.1684464047.251246<Response [200]>{'type': 'FLOW_FAILED', 'message': 'Not able to find a owning state with id c7a34000-ac8d-4d63-a0cf-ccb7b09435b2'}

In [31]:
# Transfer Test 4 -- Transfer from Non Holder 
# Expected -- Failure
(req_id, response, return_val) = h.transfer(return_val_uuid, 'Charlie', 'Bob', quantity=1)
h.message([req_id, response, return_val])

 19-05 10:41:14 | I | Tranfering instrument id 'c7a34000-ac8d-4d63-a0cf-ccb7b09435b2' to 'Bob'
 19-05 10:41:14 | I | Running action 'Transfer' on 'Charlie/AD4AA4F0D9B1/CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB'
 19-05 10:41:18 | I | {'type': 'FLOW_FAILED', 'message': 'Not able to find a owning state with id c7a34000-ac8d-4d63-a0cf-ccb7b09435b2'}
 19-05 10:41:18 | I | Time taken = 0:00:04.071231


44156.1684464074.372929<Response [200]>{'type': 'FLOW_FAILED', 'message': 'Not able to find a owning state with id c7a34000-ac8d-4d63-a0cf-ccb7b09435b2'}

In [34]:
# Query all nodes
h.query_all_nodes()

 19-05 10:43:57 | I | Result for query=ListInstrument from 'Authority/F0E569C4765C/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.104439)


,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,d4ea67e0-9945-4fe5-b0da-c3ef0d457e32,Star Alliance Travel Points,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",10,False,None,True,{}
1,c7a34000-ac8d-4d63-a0cf-ccb7b09435b2,Amazon Voucher,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",200,False,None,True,{'currency': 'INR'}


 19-05 10:44:01 | I | Result for query=ListInstrument from 'Bob/972B13CEE093/CN=Bob, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.082594)


""


 19-05 10:44:05 | I | Result for query=ListInstrument from 'Charlie/AD4AA4F0D9B1/CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.081740)


""


 19-05 10:44:09 | I | Result for query=ListInstrument from 'Alice/F07CC621F1DF/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.085795)


,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,d4ea67e0-9945-4fe5-b0da-c3ef0d457e32,Star Alliance Travel Points,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",10,False,None,True,{}
1,c7a34000-ac8d-4d63-a0cf-ccb7b09435b2,Amazon Voucher,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",200,False,None,True,{'currency': 'INR'}


### Redeem:
* Not all instruments will have expiry/maturity.
* Redeem can be executed by either the owner (holder of the asset) or also by the issuer of the asset (present design, can be controlled)
* If permitted, it will work on the basic nature of the instrument decided right from the issue point(by the issuer).
* In Asset type 1, all the redeems apart from non holders should be succeed

In Asset type 3, all the redeems apart from non holders should be **succeed**.

In [35]:
# Redeem Test1 -- Partial redeem by Holder (10 Amazon Voucher by Alice)
# Exepected -- Succeed

(req_id, response, return_val) = h.redeem('c7a34000-ac8d-4d63-a0cf-ccb7b09435b2', 'Alice', quantity=10)
h.message([req_id, response, return_val])

 19-05 10:51:29 | I | Redeeming instrument id 'c7a34000-ac8d-4d63-a0cf-ccb7b09435b2' quantity '10'
 19-05 10:51:29 | I | Running action 'Redeem' on 'Alice/F07CC621F1DF/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'
 19-05 10:52:14 | I | net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@be1e6e
 19-05 10:52:14 | I | Time taken = 0:00:44.543687


44156.1684464689.793643<Response [200]>net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@be1e6e

In [36]:
# Now, query Alice, quantity of Amazon Voucher reduced to 190
h.query('Alice')

,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,d4ea67e0-9945-4fe5-b0da-c3ef0d457e32,Star Alliance Travel Points,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",10,False,None,True,{}
1,c7a34000-ac8d-4d63-a0cf-ccb7b09435b2,Amazon Voucher,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",190,False,None,True,{'currency': 'INR'}


In [38]:
# Redeem Test 2 -- Redemption exceeded the maximum quantity by Holder 
# Exepected -- Failure
(req_id, response, return_val) = h.redeem('d4ea67e0-9945-4fe5-b0da-c3ef0d457e32', 'Alice', quantity=15)
h.message([req_id, response, return_val])

 19-05 10:58:07 | I | Redeeming instrument id 'd4ea67e0-9945-4fe5-b0da-c3ef0d457e32' quantity '15'
 19-05 10:58:07 | I | Running action 'Redeem' on 'Alice/F07CC621F1DF/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'
 19-05 10:58:11 | I | {'type': 'FLOW_FAILED', 'message': 'Redeem quantity requested is 15, whereas quantity in instrument id d4ea67e0-9945-4fe5-b0da-c3ef0d457e32 is 10 (less quantity)'}
 19-05 10:58:11 | I | Time taken = 0:00:04.081071


44156.1684465087.178814<Response [200]>{'type': 'FLOW_FAILED', 'message': 'Redeem quantity requested is 15, whereas quantity in instrument id d4ea67e0-9945-4fe5-b0da-c3ef0d457e32 is 10 (less quantity)'}

In [39]:
# Redeem Test 3 -- Partial redeem by Issuer (2 Star Alliance Travel Points by Authority)
# Exepected -- Succeed
(req_id, response, return_val) = h.redeem('d4ea67e0-9945-4fe5-b0da-c3ef0d457e32', 'Authority', quantity=2)
h.message([req_id, response, return_val])

 19-05 11:01:19 | I | Redeeming instrument id 'd4ea67e0-9945-4fe5-b0da-c3ef0d457e32' quantity '2'
 19-05 11:01:19 | I | Running action 'Redeem' on 'Authority/F0E569C4765C/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 19-05 11:02:08 | I | net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@48da0cd1
 19-05 11:02:08 | I | Time taken = 0:00:48.573322


44156.1684465279.698095<Response [200]>net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@48da0cd1

In [41]:
# Now, query Alice again, quantity of Star Alliance Travel Points reduced to 8
h.query('Alice')

,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,c7a34000-ac8d-4d63-a0cf-ccb7b09435b2,Amazon Voucher,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",190,False,None,True,{'currency': 'INR'}
1,d4ea67e0-9945-4fe5-b0da-c3ef0d457e32,Star Alliance Travel Points,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",8,False,None,True,{}


In [43]:
# Redeem Test 4 -- Full redeem by Holder (190 Amazon Voucher by Alice)
# Exepected -- Succeed
(req_id, response, return_val) = h.redeem('c7a34000-ac8d-4d63-a0cf-ccb7b09435b2', 'Alice', quantity=190)
h.message([req_id, response, return_val])

 19-05 11:07:29 | I | Redeeming instrument id 'c7a34000-ac8d-4d63-a0cf-ccb7b09435b2' quantity '190'
 19-05 11:07:29 | I | Running action 'Redeem' on 'Alice/F07CC621F1DF/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'
 19-05 11:08:06 | I | net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@4aeea55
 19-05 11:08:06 | I | Time taken = 0:00:36.366574


44156.1684465649.76219<Response [200]>net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@4aeea55

In [44]:
# Now, query Alice again, Amazon Voucher was fully redeemed
h.query('Alice')

,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,d4ea67e0-9945-4fe5-b0da-c3ef0d457e32,Star Alliance Travel Points,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",8,False,None,True,{}


In [46]:
# Now, query all
h.query_all_nodes(0)

 19-05 11:12:02 | I | Result for query=ListInstrument from 'Authority/F0E569C4765C/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.200173)


,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,d4ea67e0-9945-4fe5-b0da-c3ef0d457e32,Star Alliance Travel Points,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",8,False,None,True,{}


 19-05 11:12:07 | I | Result for query=ListInstrument from 'Bob/972B13CEE093/CN=Bob, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.084810)


""


 19-05 11:12:11 | I | Result for query=ListInstrument from 'Charlie/AD4AA4F0D9B1/CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.085468)


""


 19-05 11:12:15 | I | Result for query=ListInstrument from 'Alice/F07CC621F1DF/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.079961)


,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,d4ea67e0-9945-4fe5-b0da-c3ef0d457e32,Star Alliance Travel Points,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",8,False,None,True,{}


In [45]:
# Redeem Test 5 -- Redeem by Non-holder (2 Star Alliance Travel Points by Bob)
# Exepected -- Failure
(req_id, response, return_val) = h.redeem('d4ea67e0-9945-4fe5-b0da-c3ef0d457e32', 'Bob', quantity=2)
h.message([req_id, response, return_val])

 19-05 11:10:08 | I | Redeeming instrument id 'd4ea67e0-9945-4fe5-b0da-c3ef0d457e32' quantity '2'
 19-05 11:10:08 | I | Running action 'Redeem' on 'Bob/972B13CEE093/CN=Bob, OU=Test Dept, O=R3, L=London, C=GB'
 19-05 11:10:12 | I | {'type': 'FLOW_FAILED', 'message': 'Not able to find a owning state with id d4ea67e0-9945-4fe5-b0da-c3ef0d457e32'}
 19-05 11:10:12 | I | Time taken = 0:00:04.078159


44156.1684465808.0447118<Response [200]>{'type': 'FLOW_FAILED', 'message': 'Not able to find a owning state with id d4ea67e0-9945-4fe5-b0da-c3ef0d457e32'}

### Print to PDF:
* "Print to PDF" is used for printing any instrument with a specific format.

In [47]:
# Print to PDF Test 1 -- Print by Holder
# Expected -- Succeed
h.print_to_pdf('Alice', 'd4ea67e0-9945-4fe5-b0da-c3ef0d457e32', query=None, show=False)

 19-05 11:15:14 | I | http://localhost:9009?data={"name": "Star Alliance Travel Points", "owner": "CN=Alice, OU=Test Dept, O=R3, L=London, C=GB", "issuer": "CN=Authority, OU=Test Dept, O=R3, L=London, C=GB", "quantity": 8, "transferable": false, "expiry": null, "verifiable": true, "attributes": {}}
[WDM] - Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8.06M/8.06M [00:00<00:00, 9.30MB/s]
 19-05 11:15:19 | I | Time taken = 0:00:09.006183


'/tmp/report_d4ea67e0-9945-4fe5-b0da-c3ef0d457e32.pdf'

In [48]:
# Print to PDF Test 2 -- Print by Issuer
# Expected -- Succeed
h.print_to_pdf('Authority', 'd4ea67e0-9945-4fe5-b0da-c3ef0d457e32', query=None, show=False)

 19-05 11:16:13 | I | http://localhost:9009?data={"name": "Star Alliance Travel Points", "owner": "CN=Alice, OU=Test Dept, O=R3, L=London, C=GB", "issuer": "CN=Authority, OU=Test Dept, O=R3, L=London, C=GB", "quantity": 8, "transferable": false, "expiry": null, "verifiable": true, "attributes": {}}
 19-05 11:16:16 | I | Time taken = 0:00:07.286476


'/tmp/report_d4ea67e0-9945-4fe5-b0da-c3ef0d457e32.pdf'

In [49]:
# Print to PDF Test 3 -- Print by Non Holder
# Expected -- Failure
h.print_to_pdf('Bob', 'd4ea67e0-9945-4fe5-b0da-c3ef0d457e32', query=None, show=False)

 19-05 11:16:53 | E | No data matching id='d4ea67e0-9945-4fe5-b0da-c3ef0d457e32' in 'Bob'
